<a href="https://colab.research.google.com/github/Gru97/DeepLearningCourse/blob/main/class_sample_session4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [169]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import pandas as pd
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import datasets

In [170]:
device='cpu'
train_data_folder= 'data/FMNIST/train'
test_data_folder= 'data/FMNIST/test'
train_fmnist= datasets.FashionMNIST(train_data_folder,download=True, train=True)
test_fmnist= datasets.FashionMNIST(test_data_folder, download=True, train=False)
train_x= train_fmnist.data
train_y= train_fmnist.targets

test_x= test_fmnist.data
trest_y= test_fmnist.targets

In [171]:
train_x.shape #[60000, 28, 28]
train_y.shape #[60000]
train_x[0].shape #[28, 28] image 28*28
train_y.unique() #[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
#train_fmnist.classes

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [172]:
class ManulClassification(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1= nn.Linear(28*28,1000,True)
    self.fc2= nn.Linear(1000,10,True)  # 10 classes
    self.relu= nn.ReLU()

  def forward(self, x):
    x= self.fc1(x)
    x= self.relu(x)
    y= self.fc2(x)
    return y

In [195]:
# another way
model= nn.Sequential(
    nn.Linear(28*28, 1000),
    nn.BatchNorm1d(1000),
    nn.Dropout(0.5),
    nn.ReLU(),
    nn.Linear(1000,10)).to(device)

In [185]:
class CustomDataset(Dataset):
  def __init__(self, data, target):
    self.data = data.float()/255
    self.data= self.data.view(-1,28*28) #flatten a 2d image to 1d
    self.target=target
  def __getitem__(self, id):
    return self.data[id], self.target[id]
  def __len__(self):
    return len(self.data)

In [186]:
batch_size=32
train_dataset= CustomDataset(train_x , train_y)
validation_dataset= CustomDataset(test_x, trest_y)
train_dataset.__len__()
train_dataset.__len__()/batch_size

1875.0

In [187]:
train_dataloader= DataLoader(train_dataset,batch_size, shuffle=True)
test_dataloader= DataLoader(validation_dataset, batch_size, shuffle=True)
len(train_dataloader)  #len/batch_size

1875

In [188]:
def accuracy(output, targets):
  _, predictions = torch.max(output, 1)  # class labels of max possiblitity resulted from a neuron

  correct= (predictions==targets).sum().item()
  return correct/targets.size(0)

In [189]:
lr=0.01
#model= ManulClassification().to(device)
optimizer= torch.optim.SGD(model.parameters(), lr= lr)
loss_function= nn.CrossEntropyLoss()  # for multi class data

def train(model, dataloader, loss_function, optimizer):
  total_loss=0.0
  total_acc=0.0
  for inputs, targets in dataloader:  # len(dataloader)/batch_size = count for
    inputs, targets = inputs.to(device), targets.to(device)
    model.train()
    outputs = model(inputs)
    loss_of_a_batch = loss_function(outputs,targets)
    loss_of_a_batch.backward()
    optimizer.step()
    optimizer.zero_grad()
    total_loss += loss_of_a_batch.item()
    total_acc=accuracy(outputs,targets)
  return total_loss/len(dataloader), total_acc/len(dataloader)  #mean in every epoch

def validate(model, dataloader, loss_function):
    model.eval()
    total_loss=0.0
    total_acc=0.0
    with torch.no_grad():
      for inputs, targets in dataloader:
        inputs, targets= inputs.to(device), targets.to(device)
        outputs= model(inputs)
        loss= loss_function(outputs,targets)
        total_loss += loss.item()
        total_acc=accuracy(outputs,targets)
    return total_loss/len(dataloader), total_acc/len(dataloader)  #mean in every epoch

